### Restart and Run All Cells

### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [3]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}', 'target':'{:,.2f}','low':'{:,.2f}',
              'cost':'{:,.2f}','price':'{:,.2f}','%change':'{:,.2f}','%profit':'{:,.2f}','profit':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'cst_amt':'{:,.2f}','mkt_amt':'{:,.2f}','difa':'{:,.2f}','current':'{:,.2f}','change':'{:,.2f}',  
              'pct':'{:,.2f}%','net':'{:,.2f}','%change':'{:,.2f}%','%profit':'{:,.2f}%','sell_date':'{:%Y-%m-%d}','%decline':'{:,.2f}%'
              }
pd.options.display.max_rows = 50
#pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-10-08 2025-10-07


In [4]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
fmt_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(fmt_time)

2025-10-08 20:25:11


In [5]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
dat_path = os.path.join(base_path, "Data")
print(f"Data path (dat_path): {dat_path}") 
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [6]:
# Define a formatting function that converts a float to a string with two decimal places
def to_currency(x):
    return '฿{:,.2f}'.format(x)    

# Define a formatting function that converts an integer to a string with comma and no decimal places
def to_integer(x):
    return '{:,.0f}'.format(x)

# Define a formatting function that converts an integer to a string with comma and no decimal places
def to_percent(x):
    return '{:,.2f}%'.format(x)

# Define a formatting function that converts an integer to a string with comma and no decimal places
def to_date(x):
    return '{:%Y-%m-%d}'.format(x)

In [7]:
df_price_tdy = pd.read_csv(input_file)
df_price_tdy.dtypes
#df_price_tdy.columns.tolist()

name         object
date         object
price       float64
change      float64
%change     float64
open        float64
high        float64
low         float64
volume        int64
value(m)    float64
dtype: object

### Purchase recently sold stocks if price lower than last sold

In [9]:
#Select name from portfolio
sql = """SELECT name FROM buy"""
df_buy = pd.read_sql(sql, const)
names = df_buy.name
portfolio = ", ".join(map(lambda name: "'%s'" % name, names))
portfolio 

"'3BBIF', 'AH', 'AIMIRT', 'AWC', 'BCH', 'CPF', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'MCS', 'NER', 'ORI', 'PTG', 'PTT', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TOA', 'TVO', 'WHAIR', 'WHART'"

In [10]:
sql = """
    SELECT S.date, B.qty, B.price, B.net,  S.price, S.net, S.profit
    FROM sells S
    JOIN buys B
    ON S.buy_id = B.id 
    JOIN stocks T
    ON T.id = B.stock_id 
    WHERE T.name IN ({})
    ORDER BY S.date DESC
    LIMIT 1
""".format(portfolio)
df_tmp = pd.read_sql(sql, conpf)
df_tmp.style.format(format_dict).hide(axis="index")

date,qty,price,net,price,net,profit
2025-10-07,"3,000",26.00,"78,172.76",26.50,"79,323.91","1,151.15"


In [11]:
#
sql = """
    SELECT T.name, S.date AS sell_date, S.price AS sell_price, B.price AS buy_price, B.date AS buy_date, B.qty
    FROM sells S
    JOIN buys B
    ON S.buy_id = B.id 
    JOIN stocks T
    ON T.id = B.stock_id 
    WHERE T.name IN ({})
    ORDER BY T.name, S.date DESC
""".format(portfolio)
df_sells = pd.read_sql(sql, conpf)
df_sells.shape

(254, 6)

In [12]:
df_sells.dtypes
#df_sells.columns.tolist()

name           object
sell_date      object
sell_price    float64
buy_price     float64
buy_date       object
qty             int64
dtype: object

In [13]:
# Convert dates to datetime if they're not already
df_sells['sell_date'] = pd.to_datetime(df_sells['sell_date'])

# Get the latest record for each name
latest_records = df_sells.sort_values('sell_date', ascending=False).drop_duplicates('name')
latest_records.shape

(27, 6)

In [14]:
colu = 'name sell_date qty sell_price price low target %decline'.split()
sell_data   = pd.merge(latest_records, df_price_tdy, on='name', how='inner')
sell_data['diff'] = sell_data['price'] - sell_data['sell_price']
sell_data['%decline'] = round(sell_data['diff'] / (sell_data['sell_price']) * 100,2)
#sell_data['sell_date'] = pd.to_datetime(sell_data['sell_date'])
sell_data['target'] = round(sell_data['sell_price'] * 0.9,2)
#sell_data['amount'] = round(sell_data['target'] * sell_data['qty'])

In [15]:
# Apply filtering with both conditions
filtered = sell_data.loc[
    (sell_data['sell_price'] >= sell_data['price']),colu]

# Sort by %decline   descending
filtered = filtered.sort_values('%decline', ascending=True)

# Format display (if format_dict is defined)
#filtered.style.format(format_dict)
filtered.sample().style.format(format_dict).hide(axis="index")

name,sell_date,qty,sell_price,price,low,target,%decline
SINGER,2023-01-09,"3,600",29.25,7.05,7.00,26.32,-75.90%


In [16]:
# Convert to datetime
filtered['sell_date'] = pd.to_datetime(filtered['sell_date'])
filtered.sample(5).style.format(format_dict).hide(axis="index")

name,sell_date,qty,sell_price,price,low,target,%decline
CPNREIT,2024-09-10,"5,000",12.70,11.60,11.40,11.43,-8.66%
RCL,2025-10-07,"3,000",26.50,26.50,26.25,23.85,0.00%
AIMIRT,2022-04-18,"10,000",12.70,10.30,10.20,11.43,-18.90%
CPF,2025-09-11,"5,000",23.40,23.20,22.80,21.06,-0.85%
SCC,2022-11-07,300,340.00,218.00,215.00,306.00,-35.88%


### This year sales

In [18]:
# Apply filtering with both conditions
filtered = sell_data.loc[
    (sell_data['sell_date'].dt.year == 2025) & 
    (sell_data['sell_price'] >= sell_data['price']),
    colu]

# Sort by %decline   descending
filtered = filtered.sort_values('%decline', ascending=True)
filtered.style.format(format_dict).hide(axis="index")

name,sell_date,qty,sell_price,price,low,target,%decline
IVL,2025-07-30,"1,800",22.90,20.90,20.80,20.61,-8.73%
NER,2025-08-06,"5,000",4.52,4.24,4.24,4.07,-6.19%
JMART,2025-08-07,"3,200",9.40,9.00,9.00,8.46,-4.26%
TVO,2025-09-15,"4,000",25.75,24.90,24.90,23.18,-3.30%
GVREIT,2025-08-08,"6,000",6.70,6.50,6.45,6.03,-2.99%
ORI,2025-10-01,"10,000",2.72,2.66,2.58,2.45,-2.21%
MCS,2025-08-27,"6,000",8.40,8.25,8.20,7.56,-1.79%
CPF,2025-09-11,"5,000",23.40,23.20,22.80,21.06,-0.85%
RCL,2025-10-07,"3,000",26.50,26.50,26.25,23.85,0.00%
WHART,2025-09-15,"5,000",9.70,9.70,9.65,8.73,0.00%


In [19]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
ruby_path = os.path.join(base_path, "Ruby")
print(f"Ruby path (ruby_path): {ruby_path}") 

Ruby path (ruby_path): C:\Users\PC1\OneDrive\A5\Ruby


In [20]:
file_name = "watch-list-buys.xlsx"  
output_file = os.path.join(ruby_path, file_name)

# Apply formatting
filtered[["sell_date"]] = filtered[["sell_date"]].apply(lambda col: col.map(to_date))

# Export to Excel
filtered.sort_values('name', ascending=True).to_excel(
    output_file, 
    header=True, 
    index=False
)

### Profit per Month

In [22]:
year = 2025
month = 10
sql = '''
    SELECT name, S.date, B.qty, B.price AS buy, B.net AS cost_amt, S.price AS sell, S.net AS sale_amt, S.profit
    FROM sells S JOIN buys B ON S.buy_id = B.id
    JOIN stocks T ON B.stock_id = T.id
    WHERE YEAR(S.date) = {} AND MONTH(S.date) = {}
    ORDER BY S.date ASC, name
'''.format(year, month)

df_sells = pd.read_sql(sql, conpf)
#print(sql)

In [23]:
# Format all floats (pandas 1.4.0+)
display(df_sells.style.format({
    col: '{:,.2f}' for col in df_sells.select_dtypes(include=float).columns
}).hide(axis="index"))

name,date,qty,buy,cost_amt,sell,sale_amt,profit
ORI,2025-10-01,10000,2.62,"26,258.03",2.72,"27,139.76",881.73
RCL,2025-10-07,3000,26.00,"78,172.76",26.50,"79,323.91","1,151.15"


In [24]:
# Convert columns to interger
df_sells["qty"] = df_sells["qty"].astype(int)
# Apply formatting
df_sells[["qty"]] = df_sells[["qty"]].apply(lambda col: col.map(to_integer))
df_sells[["buy", "cost_amt","sell", "sale_amt", "profit"]] = \
df_sells[["buy", "cost_amt","sell", "sale_amt", "profit"]].apply(lambda col: col.map(to_currency))
df_sells[["date"]] = df_sells[["date"]].apply(lambda col: col.map(to_date))
df_sells.style.hide(axis="index")

name,date,qty,buy,cost_amt,sell,sale_amt,profit
ORI,2025-10-01,"10,000",฿2.62,"฿26,258.03",฿2.72,"฿27,139.76",฿881.73
RCL,2025-10-07,"3,000",฿26.00,"฿78,172.76",฿26.50,"฿79,323.91","฿1,151.15"


In [25]:
df_sells = pd.read_sql(sql, conpf)
print(f"Monthly Profit Amount: {df_sells.profit.sum():,.2f}")

Monthly Profit Amount: 2,032.88


In [26]:
# Get the current time
current_time = datetime.now()
# Format the time to remove millisAeconds
fmt_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(fmt_time)

2025-10-08 20:25:12
